In [ ]:
from pyalex import Works, invert_abstract
import pandas as pd
from sentence_transformers import SentenceTransformer


In [3]:
results = await Works().search("machine learning").get()

In [5]:
df = pd.DataFrame(results)


In [11]:
df['abstract'] = df.abstract_inverted_index.map(invert_abstract)

In [13]:
df['titleabs'] = df['title'] + '\n' + df['abstract']

In [17]:
df.titleabs.iloc[0]

'Genetic algorithms in search, optimization, and machine learning\nFrom the Publisher:\r\nThis book brings together - in an informal and tutorial fashion - the computer techniques, mathematical tools, and research results that will enable both students and practitioners to apply genetic algorithms to problems in many fields. \r\n\r\nMajor concepts are illustrated with running examples, and major algorithms are illustrated by Pascal computer programs. No prior knowledge of GAs or genetics is assumed, and only a minimum of computer programming and mathematics background is required.'